In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import plotly_express as px
from dash.dependencies import Input, Output
rootfold = '/home/j0hndoe/Documents/git/reddit-disinformation'

In [2]:
def get_flag_perc(df, groups):
    return df.\
        groupby(groups)\
            [['n_subm_kept','ISFLAG']].\
                agg(sum).sort_index().reset_index().\
        assign(perc_FLAG = lambda df: df.ISFLAG/df.n_subm_kept)

In [3]:
df_flags = pd.read_csv(rootfold+"/dashboard/data/app_flags_large.csv")
df_day = pd.read_csv(rootfold+"/dashboard/data/app_subr_day.csv")

df_day['week'] = df_day['week'].astype('int')
min_week, max_week = df_day['week'].min(), df_day['week'].max()
weekrange = [min_week, max_week]

filtered_df_day = df_day[(df_day['week'].between(*weekrange))]# & (df['varr']==varr)]
filtered_df_day['ISFLAG'] = filtered_df_day.flag.copy()#loc[:,flagtypes].sum(axis=1)>0
### Aggregate
filtered_df_week = get_flag_perc(
    filtered_df_day,
    groups = ['subreddit_cat','subreddit','week'])
filtered_df_agg = get_flag_perc(
    filtered_df_week,
    groups = ['subreddit_cat','subreddit'])

In [4]:
df_flags

,subreddit,subm_id,comm_id,month,week,day,subreddit_cat,sent_id,sent,flag,...,unreliable,propaganda,other,comm_parent_id,subm_title,subm_removed,subm_covid,subm_link,link_url,subm_ncomments
0,nottheonion,t3_gjysno,t1_fqnw4f5,5.0,20.0,2020-05-15,generic_news,7,"well, it's fake news hiding behind the ""satire...",1,...,0,0,0,t3_gjysno,Experts Worry Unreal Engine 5 Will Deplete the...,deleted,False,https://www.reddit.com/r/nottheonion/comments/...,https://thehardtimes.net/harddrive/experts-wor...,2.0
1,nottheonion,t3_gjysim,t1_fqnw3h5,5.0,20.0,2020-05-15,generic_news,7,"well, it's fake news hiding behind the ""satire...",1,...,0,0,0,t3_gjysim,We’re running out of Triangles !.!.!,deleted,False,https://www.reddit.com/r/nottheonion/comments/...,https://thehardtimes.net/harddrive/experts-wor...,2.0
2,LosAngeles,t3_gjtxlv,t1_fqn2uq8,5.0,20.0,2020-05-14,local,0,Super misleading headline.,1,...,0,0,0,t3_gjtxlv,Unemployment in CA expected to peak at nearly ...,deleted,False,https://www.reddit.com/r/LosAngeles/comments/g...,https://losangeles.cbslocal.com/2020/05/14/cor...,2.0
3,bayarea,t3_gjokcl,t1_fqmxo1j,5.0,20.0,2020-05-14,local,0,This title is extremely misleading,1,...,0,0,0,t3_gjokcl,"CA schools may reopen with mask requirement, s...",visible,False,https://www.reddit.com/r/bayarea/comments/gjok...,https://www.kcra.com/article/ca-schools-may-re...,92.0
4,AmericanPolitics,t3_gjrjpf,t1_fqmphx8,5.0,20.0,2020-05-14,generic_politics,0,More anti-vaxxer propaganda.,1,...,0,1,0,t3_gjrjpf,Niall McCrae and David Kurten: EU Numbers Show...,visible,True,https://www.reddit.com/r/AmericanPolitics/comm...,https://www.thegatewaypundit.com/2020/05/niall...,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27388,politics,t3_eieo7b,t1_fcpcw7f,1.0,1.0,2020-01-01,generic_politics,1,it's bullshit,1,...,0,0,0,t3_eieo7b,Giuliani Says He’s Prepared to ‘Do Demonstrati...,visible,False,https://www.reddit.com/r/politics/comments/eie...,https://www.thedailybeast.com/giuliani-says-he...,54.0
27389,politics,t3_eid766,t1_fcqg7lz,1.0,1.0,2020-01-01,generic_politics,2,this is such clickbait,1,...,0,0,0,t3_eid766,Trump follows Iran threat with bizarre 'change...,visible,False,https://www.reddit.com/r/politics/comments/eid...,https://www.independent.co.uk/news/world/ameri...,212.0
27390,politics,t3_eicy4q,t1_fcqtl95,1.0,1.0,2020-01-01,generic_politics,1,This is bullshit propaganda and if he for some...,1,...,0,1,0,t3_eicy4q,Giuliani says he would be willing to testify i...,visible,False,https://www.reddit.com/r/politics/comments/eic...,https://thehill.com/homenews/administration/47...,811.0
27391,politics,t3_eicy4q,t1_fcpo47c,1.0,1.0,2020-01-01,generic_politics,0,It's all the same bullshit it starts with the ...,1,...,0,0,0,t3_eicy4q,Giuliani says he would be willing to testify i...,visible,False,https://www.reddit.com/r/politics/comments/eic...,https://thehill.com/homenews/administration/47...,811.0


In [14]:
filtered_df_week_cat = get_flag_perc(filtered_df_week, ['subreddit_cat','week'])
subr_line_all = px.line(
                    filtered_df_week_cat,
                    x = "week",
                    y = "ISFLAG",
                    color = "subreddit_cat")
subr_line_all.show()

In [9]:
fig_subr_treemap = px.treemap(filtered_df_agg, 
                      path=['subreddit_cat', 'subreddit'], 
                      values='ISFLAG',
                      color='subreddit_cat')
fig_subr_treemap.show()